Classification Model

In [69]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

In [70]:
df_pluviometricos = pd.read_csv("../dados/dados_pluviometricos_geo_anos.csv")

In [71]:
df_curva_niveis = pd.read_csv("../dados/dados_curva_niveis.csv")

In [72]:
df_curva_niveis = df_curva_niveis[["score", "hex", "geometry", "Elevation"]]

In [73]:
df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_24491/975208766.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_curva_niveis = df_curva_niveis.groupby(["hex", "geometry"], as_index=False)["Elevation", "score"].mean()


Preparar o dataset de alagamentos (o target)

In [74]:
df_alagamentos = pd.read_csv("../dados/alagamentos.csv")

In [75]:
df_alagamentos["flooded"] = df_alagamentos["NATUREZA"]

In [76]:
df_alagamentos["date"] = df_alagamentos["DATA"]
df_alagamentos = df_alagamentos[["flooded", "date", "hex", "NOME", "geometry"]]

In [77]:

df_alagamentos.date = df_alagamentos.date.astype('datetime64[ns]')

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_24491/3419792466.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_alagamentos.date = df_alagamentos.date.astype('datetime64[ns]')


In [78]:
df_alagamentos['date'] = df_alagamentos['date'].dt.strftime('%Y-%m-%d')

Preparar a elevação

In [79]:
df_merged = pd.merge(df_alagamentos, df_curva_niveis, on="geometry", how="outer")

In [80]:
df_merged["elevation"] = df_merged["Elevation"]
df_merged["hex"] = df_merged["hex_x"]
df_merged["score_elevation"] = df_merged["score"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation"]]

In [81]:
df_merged.drop_duplicates(inplace=True)

In [82]:
df_merged = pd.merge(df_pluviometricos, df_merged, on=["geometry", "date"], how="left")

In [83]:
df_merged = pd.merge(df_merged, df_curva_niveis, on=["geometry"], how="left")

In [84]:

df_merged["value_precipitation"] = df_merged["valorMedido"]
df_merged["value_precipitation_3_days"] = df_merged["value_last_3_days"]
df_merged["value_precipitation_7_days"] = df_merged["value_last_7_days"]
df_merged["hex"] = df_merged["hex_x"]
df_merged = df_merged[["flooded", "date", "hex", "geometry", "score_elevation", "elevation", "value_precipitation", "value_precipitation_3_days", "value_precipitation_7_days"]]

In [85]:
df_merged = pd.merge(df_merged, df_curva_niveis[['geometry', 'score', 'Elevation']], on='geometry', how='left')

In [86]:
df_merged['score_elevation'].fillna(df_merged['score'], inplace=True)
df_merged['elevation'].fillna(df_merged['Elevation'], inplace=True)
df_merged['flooded'].fillna("Nenhum", inplace=True)

In [87]:
df_merged.drop(columns='score', inplace=True)
df_merged.drop(columns='Elevation', inplace=True)

In [88]:
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month
df_merged['day'] = df_merged['date'].dt.day

In [98]:
df_merged.drop_duplicates(inplace=True)

In [99]:
df_merged[df_merged["flooded"] == "Alagamento"]

,flooded,date,hex,geometry,score_elevation,elevation,value_precipitation,value_precipitation_3_days,value_precipitation_7_days,year,month,day
9062,Alagamento,2015-10-22,88a804cb41fffff,POLYGON ((-49.33694002968239 -25.4052233804035...,99.5,967.049180,60.544994,52.215987,54.987237,2015,10,22
13445,Alagamento,2015-10-21,88a804cb49fffff,POLYGON ((-49.34023347844297 -25.4124704980994...,99.5,953.672131,59.109061,0.000000,1.403529,2015,10,21
34189,Alagamento,2015-08-02,88a804d86dfffff,POLYGON ((-49.355572751255856 -25.503956849950...,96.0,920.290698,0.000000,0.000000,0.294974,2015,8,2
36158,Alagamento,2017-12-22,88a804d905fffff,POLYGON ((-49.309048293845166 -25.478700437163...,100.0,915.345455,41.254045,9.868296,25.355981,2017,12,22
36159,Alagamento,2017-12-23,88a804d905fffff,POLYGON ((-49.309048293845166 -25.478700437163...,100.0,915.345455,4.402516,51.122341,51.050135,2017,12,23
...,...,...,...,...,...,...,...,...,...,...,...,...
400637,Alagamento,2016-08-20,88a831adb1fffff,POLYGON ((-49.2122644072129 -25.49699169415437...,99.5,892.905660,56.589717,51.774454,63.194167,2016,8,20
401746,Alagamento,2016-09-02,88a831adbbfffff,POLYGON ((-49.2214295394814 -25.49788388919803...,99.5,878.187500,0.204890,12.220728,12.285600,2016,9,2
422618,Alagamento,2016-10-20,88a831ae95fffff,POLYGON ((-49.3119299050174 -25.56199807088341...,98.0,920.306306,29.058548,81.846152,123.845441,2016,10,20
424404,Alagamento,2015-09-10,88a831ae9bfffff,POLYGON ((-49.330254284209296 -25.563764979069...,99.5,893.704918,19.291447,49.208366,74.716444,2015,9,10


Rodar modelo de classificação

In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

features = ['elevation', 'score_elevation', 'value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days', "date"]
target = 'flooded'

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_merged[features], df_merged[target], test_size=0.2, random_state=42)

# Criar e treinar a árvore de decisão
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Avaliar o desempenho do modelo no conjunto de teste
score = clf.score(X_test, y_test)
print("Acurácia do modelo:", score)


In [101]:
features_elevation = ['elevation', 'score_elevation']
features_precipitation = ['value_precipitation', 'value_precipitation_3_days', 'value_precipitation_7_days']
date = ["year", "month", "day"]

In [102]:
features = df_merged[features_elevation+features_precipitation+date]
target = df_merged[['flooded']]

In [107]:
train_features, test_features, train_labels, test_labels = train_test_split(features, target, test_size=0.2)

In [104]:
rf = RandomForestClassifier()
rf.fit(train_features, train_labels)

/var/folders/ld/8dhf4gy93xbd1ybmf6nljm38dn1p1r/T/ipykernel_24491/2852299628.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_features, train_labels)


RandomForestClassifier()

In [105]:
predictions = rf.predict(test_features)

In [106]:
print("Accuracy: " + str(accuracy_score(test_labels, predictions)))
print("Recall for nothing: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Nenhum")))
print("Recall for flooded: " + str(recall_score(test_labels, predictions, average="binary", pos_label="Alagamento")))
print(confusion_matrix(test_labels, predictions))

Accuracy: 0.9996421926434808
Recall for nothing: 1.0
Recall for flooded: 0.0
[[     0     80]
 [     0 223504]]


In [108]:
print("Len: " + str(len(predictions)))
alagamentos = []
nenhum = []
for f in predictions:
    if f == "Nenhum":
        nenhum.append(f)
    if f == "Alagamento":
        alagamentos.append(f)

print("Len alagamentos: " + str(len(alagamentos)))
print("Len nenhum: " + str(len(nenhum)))

Len: 223584
Len alagamentos: 0
Len nenhum: 223584


Verificar a porcentagem de flooded

In [115]:

total_alagamentos = len(df_merged[df_merged['flooded'] == "Alagamento"])
total_nenhum = len(df_merged[df_merged['flooded'] == "Nenhum"])

print("Alagamentos: " + str(total_alagamentos))
print("Nenhum: " + str(total_nenhum))
print("Porcentagem: " + str(100*total_alagamentos/(total_alagamentos+total_nenhum)) + "%")

Alagamentos: 153
Nenhum: 447015
Porcentagem: 0.034215328467153285%
